In [ ]:
!pip install ultralytics

In [ ]:
!pip install opencv-python

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="coco128.yaml", epochs=3)  # train the model
metrics = model.val()  # evaluate model performance on the validation set
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
success = model.export(format="onnx")  # export the model to ONNX format

In [2]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
results= model("https://ultralytics.com/images/bus.jpg")


100%|██████████| 476k/476k [00:00<00:00, 5.12MB/s]
image 1/1 C:\Users\user\Documents\machine_learning\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 24.0ms
Speed: 1.0ms preprocess, 24.0ms inference, 7.7ms postprocess per image at shape (1, 3, 640, 640)


In [5]:
from PIL import Image
image= Image.open("bus.jpg")
results= model.predict(source=image, save=True)


0: 640x480 4 persons, 1 bus, 1 stop sign, 56.0ms
Speed: 190.3ms preprocess, 56.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict


In [3]:
!yolo mode=predict model=yolov8n.pt source=0

Ultralytics YOLOv8.0.66 🚀 Python-3.9.12 torch-2.0.0 CPU
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

[ WARN:0@12.953] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
Traceback (most recent call last):
  File "/Users/chanifrusydi/miniconda3/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/Users/chanifrusydi/miniconda3/lib/python3.9/site-packages/ultralytics/yolo/cfg/__init__.py", line 391, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/Users/chanifrusydi/miniconda3/lib/python3.9/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/Users/chanifrusydi/miniconda3/lib/python3.9/site-packages/ultralytics/yolo/engine/model.py", line